In [1]:
import os 
import os.path as osp
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import random
import open3d as o3d
import open3d.core as o3c
import json
import subprocess
import pickle
from  plyfile import PlyData
import torch
from tqdm import tqdm
import torch.multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
#the other imports from the local stuff
import sys

ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r,visualisation

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/local/home/ekoller/BT


In [129]:
#reading in the necessary data
data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
scan_id= "10b1795b-3938-2467-88fe-b10cad8913f8" #is reference scan  since it is a reference scan everything shouls be correctly hit
frame_number = "000033"
img_width = 960
img_height = 540

curr_scan_id = "4a9a43e6-7736-2874-8723-08ae98eccd16" #is ref 1d2f8510-d757-207c-8c48-3684433860e1
new_scan_id =  "4a9a43e8-7736-2874-8519-29915757c372" #is rescan 9c27de56-6184-2cda-8196-591957b6387d

#the original meshes are given in the file  'labels.instances.annotated.v2.ply'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(torch.cuda.is_available() )



This code segment does the following: given a mesh of the scene, create the camera rays and intersect them with the mesh- For every camera pose do the following: the camera pose is given in the original frame of the scan: if reference scan in reference coordinates, if rescan in rescan coordinates. To do the computations the designdecision is the following:  transform the "current" mesh into the coordinate system of the "new mesh". in this coordinate system you can then compare the two things. Compare the object of the intersection of the current mesh with the one of the new mesh


Logic for the try out data: in the gt_tmp we have for every scene a folder with the frame of the ground truth projection
in the projection file we have the following way of storing things projection/curr_scene_id/(color/global_id/obj_id)/projection_pose_in_new_scene_id_000033

In [22]:
#transform the mesh based on the procided matrix, mesh is given as ply file
def transform_mesh(curr_scan_id, new_scan_id, matrix):
    #read the file
    mesh_path = osp.join(data_dir,"scenes", curr_scan_id, 'labels.instances.annotated.v2.ply')
    with open(mesh_path, 'rb') as file:
        plydata = PlyData.read(file)
    #transform the points into the correct coordinate system using homog coordinates
    points = np.stack((plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z'])).transpose()
    points4f = np.insert(points, 3, values=1, axis=1)
    points = points4f * matrix

    #put them back into 3d points
    plydata['vertex']['x'] = np.asarray(points[:,0]).flatten()
    plydata['vertex']['y'] = np.asarray(points[:,1]).flatten()
    plydata['vertex']['z'] = np.asarray(points[:,2]).flatten()

    #needs to be stored somewho
    filename = "transformation_from_" + str(curr_scan_id) + "_to_" + str(new_scan_id) + ".ply"
    output_path = osp.join(data_dir,"tmpfiles", curr_scan_id, filename)
    directory = osp.dirname(output_path)
    os.makedirs(directory, exist_ok=True)

    plydata.write(output_path)

    return output_path


In [102]:
# transform the current mesh into the coordinate system of the assuming the original meshes in original coordinate systems, returns the path to the new file 
#with the transformed coordinates
def transform_to_coordinates(data_dir, curr_scan_id, new_scan_id):

    #get the path to the matricies of each scan_id for transformation of rescan to reference
    path = osp.join(data_dir,"files", "3RScan.json")
    rescan2ref_all_id = scan3r.read_transform_mat(path)

    #either both reference or the same id -> same coordinate system already
    if curr_scan_id == new_scan_id:
        mesh_path = osp.join(data_dir,"scenes", curr_scan_id, 'labels.instances.annotated.v2.ply')
        with open(mesh_path, 'rb') as file:
            plydata = PlyData.read(file)
          #needs to be stored somewho
        filename = "transformation_from_" + str(curr_scan_id) + "_to_" + str(new_scan_id) + ".ply"
        output_path = osp.join(data_dir,"tmpfiles", curr_scan_id, filename)
        directory = osp.dirname(output_path)
        os.makedirs(directory, exist_ok=True)

        plydata.write(output_path)
        return output_path
    

    # current is reference, new is rescan -> simple transformation 
    elif scan3r.is_reference(data_dir, curr_scan_id):
        #get the rescantore for the new scan id
        rescan2ref = rescan2ref_all_id[new_scan_id]
        rescan2ref = rescan2ref.reshape(4,4)
        ref2rescan = np.linalg.inv(rescan2ref)
        return transform_mesh(curr_scan_id, new_scan_id, ref2rescan)
    
    # current is rescan, new is reference -> simple transformation
    elif scan3r.is_reference(data_dir, new_scan_id):
        rescan2ref = rescan2ref_all_id[curr_scan_id]
        rescan2ref = rescan2ref.reshape(4,4)
        return transform_mesh(curr_scan_id, new_scan_id,rescan2ref)
    
    #both are rescans but different rescans
    else:
        assert(scan3r.is_rescan(data_dir, curr_scan_id))
        assert(scan3r.is_rescan(data_dir, new_scan_id))

        #transformation matrix from current rescan coord to ref coordinates
        rescan2ref_1 = rescan2ref_all_id[curr_scan_id]
        rescan2ref_1 = rescan2ref_1.reshape(4,4)

        #transformation matrix from ref coord to new rescan coordinates
        rescan2ref_2 = rescan2ref_all_id[new_scan_id]
        rescan2ref_2 = rescan2ref_2.reshape(4,4)
        ref2rescan_2 = np.linalg.inv(rescan2ref_2)

        #combine the 2 transformations first_1 then _2 so from right to left matrix multiplication
        rescan2rescan = np.dot(rescan2ref_1,ref2rescan_2) #potential bug fix
        return transform_mesh(curr_scan_id,new_scan_id, rescan2rescan)


In [24]:
#the function which actually does the intersection part
def segmentResult(scene, intrinsics, extrinsics, width, height,
                      mesh_triangles, num_triangles, colors, obj_ids, global_ids):
        
        #create the rays to shoot in the scene
        rays = o3d.t.geometry.RaycastingScene.create_rays_pinhole(
            intrinsic_matrix = intrinsics.astype(np.float64),
            extrinsic_matrix = extrinsics.astype(np.float64),
            width_px = width, height_px = height
        )
        
        #find the intersections of the rays
        ans = scene.cast_rays(rays)
        hit_triangles_ids = ans['primitive_ids'].numpy()
        hit_triangles_ids_valid_masks = (hit_triangles_ids<num_triangles)
        hit_triangles_ids_valid = hit_triangles_ids[hit_triangles_ids_valid_masks]
        hit_triangles_valid = mesh_triangles[hit_triangles_ids_valid]
        hit_points_ids_valid = hit_triangles_valid[:,0]
        
        #create resulting maps initialized to 0
        color_map = np.zeros((height,width,3), dtype=np.uint8)
        obj_id_map = np.zeros((height,width), dtype=np.uint8)
        global_id_map = np.zeros((height,width), dtype=np.uint8)

        #go through the array and add the corresponding values
        color_map[hit_triangles_ids_valid_masks] = colors[hit_points_ids_valid]
        obj_id_map[hit_triangles_ids_valid_masks] = obj_ids[hit_points_ids_valid]
        global_id_map[hit_triangles_ids_valid_masks] = global_ids[hit_points_ids_valid]
        
        return color_map, obj_id_map, global_id_map

In [29]:
#given a scan id and a corresponding frame number in the same coordinate system sanity check if the same as groundtruth
def project_mesh_gt_generation(data_dir,scenes_dir, scan_id, frame_number):
        # get related files
        mesh_file = osp.join(scenes_dir, scan_id, "labels.instances.annotated.v2.ply")
        

        # get img info and camera intrinsics 
        camera_info = scan3r.load_intrinsics(scenes_dir, scan_id)
        intrinsics = camera_info['intrinsic_mat']
        img_width = int(camera_info['width'])
        img_height = int(camera_info['height'])
        
        # load labels
        plydata_npy = np.load(osp.join(scenes_dir, scan_id, "data.npy"), allow_pickle=True)
        
        #print the names of the datafields
        # if plydata_npy.dtype.names:
        #         for name in plydata_npy.dtype.names:
        #              print(f"Field: {name}")
        #              print(plydata_npy[name])
        # else:
        #     print(plydata_npy)


        #get the corresponding ids
        obj_labels = plydata_npy['objectId']
        global_labels = plydata_npy['globalId']
    
      
    
        # load mesh and scene
        mesh = o3d.io.read_triangle_mesh(mesh_file)
        mesh_triangles = np.asarray(mesh.triangles)
        #also load the colour
        colors = np.asarray(mesh.vertex_colors)*255.0
        colors = colors.round()
        num_triangles = mesh_triangles.shape[0]
        scene = o3d.t.geometry.RaycastingScene()
        scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh))
        
        # get frame_indexes
        frame_idxs = frame_number
        poses = scan3r.load_pose(scenes_dir, scan_id, frame_number)
        
        # project 3D model using rays
        obj_id_imgs = {}
        global_id_imgs = {}
        color_imgs = {}
        for idx in range(0, len(poses), 1):
            #frame_idx = frame_idxs[idx] #adjusted for single use in this jupiter notebook
            frame_idx = frame_idxs
            #img_pose = poses[idx] #adjusted for single use in this jupiter notebook
            img_pose = poses
            img_pose_inv = np.linalg.inv(img_pose)
            #get the mappings for the image
            color_map, obj_id_map, global_id_map = segmentResult(
                scene, intrinsics, img_pose_inv, img_width, img_height, 
                mesh_triangles, num_triangles, colors, obj_labels, global_labels
            )
            obj_id_imgs[frame_idx] = obj_id_map
            global_id_imgs[frame_idx] = global_id_map
            color_imgs[frame_idx] = color_map
            
        #create directories (here tmp to try out & save 
        save_scan_color_dir = osp.join(data_dir, "gt_tmp", scan_id, "color")
        save_scan_obj_dir = osp.join(data_dir, "gt_tmp", scan_id, "obj_id")
        save_scan_global_dir = osp.join(data_dir, "gt_tmp", scan_id, "global_id")

        #make sure the directories exist
        for dir_path in [save_scan_color_dir, save_scan_obj_dir, save_scan_global_dir]:
            try:
                os.makedirs(dir_path, exist_ok=True)
            except Exception as e:
                print(f"Failed to create directory {dir_path}: {e}")

        
        for frame_idx in obj_id_imgs:
            obj_id_img = obj_id_imgs[frame_idx]
            global_id_img = global_id_imgs[frame_idx]
            color_img = color_imgs[frame_idx]
            
            
            img_name = "gt_frame-"+str(frame_idx)+".jpg"
            obj_id_img_file = osp.join(save_scan_obj_dir, img_name)
            global_img_file = osp.join(save_scan_global_dir, img_name)
            color_img_file = osp.join(save_scan_color_dir, img_name)
            success_obj = cv2.imwrite(obj_id_img_file, obj_id_img)
            success_global = cv2.imwrite(global_img_file, global_id_img)
            success_color = cv2.imwrite(color_img_file, color_img)

            #also vusually double check if the images are correct
            # cv2.imshow("object_id", obj_id_img)
            # cv2.imshow("global_id", global_id_img)
            # cv2.imshow("color", color_img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()





In [30]:
#try out the ground truth generation
project_mesh_gt_generation(data_dir,scenes_dir, scan_id, frame_number)


In [51]:
#given a current scene and a new scene: transforma the current scene into the coordinate system of the new scene and perform
#raytracing based on the new camera pose onto the current mesh which is transformed into the coordinate system of the new scene
#frame number is for the new_scan_id
#returns numpy arrays of object_id, color, global_id
def project_mesh_diff(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number):

    """
    Read/prepare the data of the current mesh (mesh, colours, objid, globalid)
    """
    #get the current coordinate system in the new coordinate system
    mesh_file = transform_to_coordinates(data_dir, curr_scan_id, new_scan_id)


    # load labels based on current mesh "status quo"
    plydata_npy = np.load(osp.join(scenes_dir, curr_scan_id, "data.npy"), allow_pickle=True)
      
    obj_labels = plydata_npy['objectId']
    global_labels = plydata_npy['globalId']
    

    # load curr mesh and scene
    mesh = o3d.io.read_triangle_mesh(mesh_file)
    mesh_triangles = np.asarray(mesh.triangles)
    #also load the colour
    colors = np.asarray(mesh.vertex_colors)*255.0
    colors = colors.round()
    num_triangles = mesh_triangles.shape[0]
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(mesh))


    """
    Read/prepare the data of the new mesh (pose, intrinsics)
    """
    # get img info and camera intrinsics of the new img
    camera_info = scan3r.load_intrinsics(scenes_dir, new_scan_id)
    intrinsics = camera_info['intrinsic_mat']
    img_width = int(camera_info['width'])
    img_height = int(camera_info['height'])

    # get frame_indexes and pose of the new
    frame_idxs = frame_number
    poses = scan3r.load_pose(scenes_dir, new_scan_id, frame_number)


    """
    Do the intersection
    """

    # project 3D model using rays
    obj_id_imgs = {}
    global_id_imgs = {}
    color_imgs = {}
    for idx in range(0, len(poses), 1):
        #frame_idx = frame_idxs[idx] #adjusted for single use in this jupiter notebook
        frame_idx = frame_idxs
        #img_pose = poses[idx] #adjusted for single use in this jupiter notebook
        img_pose = poses
        img_pose_inv = np.linalg.inv(img_pose)
        #get the mappings for the image
        color_map, obj_id_map, global_id_map = segmentResult(
            scene, intrinsics, img_pose_inv, img_width, img_height, 
            mesh_triangles, num_triangles, colors, obj_labels, global_labels
        )
        obj_id_imgs[frame_idx] = obj_id_map
        global_id_imgs[frame_idx] = global_id_map
        color_imgs[frame_idx] = color_map
        
    #create directories (here tmp to try out & save 
    save_scan_color_dir = osp.join(data_dir, "projection", curr_scan_id, "color")
    save_scan_obj_dir = osp.join(data_dir, "projection", curr_scan_id, "obj_id")
    save_scan_global_dir = osp.join(data_dir, "projection", curr_scan_id, "global_id")

    #make sure the directories exist
    for dir_path in [save_scan_color_dir, save_scan_obj_dir, save_scan_global_dir]:
        try:
            os.makedirs(dir_path, exist_ok=True)
            
        except Exception as e:
            print(f"Failed to create directory {dir_path}: {e}")

    
    for frame_idx in obj_id_imgs:
        obj_id_img = obj_id_imgs[frame_idx]
        global_id_img = global_id_imgs[frame_idx]
        color_img = color_imgs[frame_idx]

        
        img_name = "projection_pose_in_"+ str(new_scan_id)+"_"+str(frame_idx)+".jpg"
        obj_id_img_file = osp.join(save_scan_obj_dir, img_name)
        global_img_file = osp.join(save_scan_global_dir, img_name)
        color_img_file = osp.join(save_scan_color_dir, img_name)
        success_obj = cv2.imwrite(obj_id_img_file, obj_id_img)
        success_global = cv2.imwrite(global_img_file, global_id_img)
        success_color = cv2.imwrite(color_img_file, color_img)

        #also vusually double check if the images are correct
        # cv2.imshow("object_id", obj_id_img)
        # cv2.imshow("global_id", global_id_img)
        # cv2.imshow("color", color_img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()   

    
    

In [34]:
project_mesh_diff(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number)

In [35]:
#this code returns the colour segmentation of the gt projection on pixelwise level
def load_gt_color_annos(data_dir, scan_id, skip=None):
    anno_imgs = {}
    frame_idxs = scan3r.load_frame_idxs(osp.join(data_dir, "scenes"), scan_id, skip)
    anno_folder = osp.join(data_dir, "files", 'gt_projection/color', scan_id)
    for frame_idx in frame_idxs:
        anno_color_file = osp.join(anno_folder, "frame-{}.jpg".format(frame_idx))
        anno_color = cv2.imread(anno_color_file, cv2.IMREAD_UNCHANGED)
        anno_imgs[frame_idx] = anno_color
    return anno_imgs

In [36]:
#this code returns the object_id segmentation of the gt projection on pixelwise level
def load_gt_obj_id_annos(data_dir, scan_id, skip=None):
    anno_imgs = {}
    frame_idxs = scan3r.load_frame_idxs(osp.join(data_dir, "scenes"), scan_id, skip)
    anno_folder = osp.join(data_dir, "files", 'gt_projection/obj_id', scan_id)
    for frame_idx in frame_idxs:
        anno_obj_file = osp.join(anno_folder, "frame-{}.jpg".format(frame_idx))
        anno_obj = cv2.imread(anno_obj_file, cv2.IMREAD_UNCHANGED)
        anno_imgs[frame_idx] = anno_obj
    return anno_imgs

In [37]:
#this code returns the colour segmentation of the gt projection on pixelwise level
def load_gt_gloabl_id_annos(data_dir, scan_id, skip=None):
    anno_imgs = {}
    frame_idxs = scan3r.load_frame_idxs(osp.join(data_dir, "scenes"), scan_id, skip)
    anno_folder = osp.join(data_dir, "files", 'gt_projection/global_id', scan_id)
    for frame_idx in frame_idxs:
        anno_global_file = osp.join(anno_folder, "frame-{}.jpg".format(frame_idx))
        anno_global = cv2.imread(anno_global_file, cv2.IMREAD_UNCHANGED)
        anno_imgs[frame_idx] = anno_global
    return anno_imgs

In [127]:
#this function gives the transformation on a pixelwise level between the current mesh and the new mesh
#this method goes over the projection and the semantically segmented image provided as input from (new_scan_id)
#it stores the mask following the logic described above
def get_transformation_mask(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number):

    #the ground truth of the new scan id must exist already
    project_mesh_gt_generation(data_dir,scenes_dir, new_scan_id, frame_number)


    #get the information of the rgb image of new_scan_id based on the ground truth (this code is for after the ground truth has beeen generated)
    # gt_colors = load_gt_color_annos(data_dir,new_scan_id)
    # gt_obj_ids = load_gt_obj_id_annos(data_dir, new_scan_id)
    # gt_global_ids = load_gt_gloabl_id_annos(data_dir,new_scan_id)

    # #get the infor for the specific frame number
    # gt_color = gt_colors[frame_number]
    # gt_obj_id = gt_obj_ids[frame_number]
    # gt_global_id= gt_global_ids[frame_number]
    scan_color_file = osp.join(data_dir, "gt_tmp", new_scan_id, "color","gt_frame-{}.jpg".format(frame_number))
    scan_obj_file = osp.join(data_dir, "gt_tmp", new_scan_id, "obj_id","gt_frame-{}.jpg".format(frame_number))
    scan_global_file = osp.join(data_dir, "gt_tmp", new_scan_id, "global_id","gt_frame-{}.jpg".format(frame_number))

    gt_color = cv2.imread(scan_color_file, cv2.IMREAD_UNCHANGED)
    gt_obj_id = cv2.imread(scan_obj_file, cv2.IMREAD_UNCHANGED)
    gt_global_id= cv2.imread(scan_global_file, cv2.IMREAD_UNCHANGED)



    #get the rayintersection of the mesh based on the new computations
    #compute the intersections and maps
    project_mesh_diff(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number)
    color_path = osp.join(data_dir,"projection",curr_scan_id, "color","projection_pose_in_{}_{}.jpg".format(new_scan_id,frame_number))
    obj_path = osp.join(data_dir,"projection",curr_scan_id, "obj_id","projection_pose_in_{}_{}.jpg".format(new_scan_id,frame_number))
    gloabl_path = osp.join(data_dir,"projection",curr_scan_id, "global_id","projection_pose_in_{}_{}.jpg".format(new_scan_id,frame_number))

    #access the files
    pj_color = cv2.imread(color_path, cv2.IMREAD_UNCHANGED)
    pj_obj_id = cv2.imread(obj_path, cv2.IMREAD_UNCHANGED)
    pj_global_id = cv2.imread(gloabl_path, cv2.IMREAD_UNCHANGED)


    #iterate through the whole image 
    res_color = np.zeros_like(gt_color)
    res_obj_id = np.zeros_like(gt_color)
    res_global_id = np.zeros_like(gt_color)

    #make the mask: originally black: wherever there is a difference take the value of the new_scan_id gt
    for i in range(img_height):
        for j in range(img_width):
            if (pj_obj_id[i][j] != gt_obj_id[i][j]):
                res_color[i][j] = gt_color[i][j]
                res_obj_id[i][j] = gt_obj_id[i][j]
                res_global_id[i][j] = gt_global_id[i][j]


    #also save the files as needed
    #create directories (here tmp to try out & save 
    save_mask_color_dir = osp.join(data_dir, "mask", curr_scan_id, "color")
    save_mask_obj_dir = osp.join(data_dir, "mask", curr_scan_id, "obj_id")
    save_mask_global_dir = osp.join(data_dir, "mask", curr_scan_id, "global_id")

    #make sure the directories exist
    for dir_path in [save_mask_color_dir, save_mask_obj_dir, save_mask_global_dir]:
        try:
            os.makedirs(dir_path, exist_ok=True)
            
        except Exception as e:
            print(f"Failed to create directory {dir_path}: {e}")

    #save the files in the corresponding directories
        
    img_name = "mask_pose_in_"+ str(new_scan_id)+"_"+str(frame_number)+".jpg"
    obj_id_img_file = osp.join(save_mask_obj_dir, img_name)
    global_img_file = osp.join(save_mask_global_dir, img_name)
    color_img_file = osp.join(save_mask_color_dir, img_name)
    success_obj = cv2.imwrite(obj_id_img_file, res_obj_id)
    success_global = cv2.imwrite(global_img_file, res_global_id)
    success_color = cv2.imwrite(color_img_file, res_color)

    return res_color,res_obj_id,res_global_id




In [130]:
#computes the mask and shows the mask
color,obj_id,global_id =get_transformation_mask(data_dir, scenes_dir,curr_scan_id, new_scan_id, frame_number)
cv2.imshow("object_id", obj_id)
cv2.imshow("global_id", global_id)
cv2.imshow("color", color)
cv2.waitKey(0)
cv2.destroyAllWindows()  


In [133]:
#sanity check if the transformations overall actually do what they are supposed to do using visuals

"""
Curr Mesh preparation and display adding
"""
pathToMesh = osp.join(data_dir,"scenes", curr_scan_id, "labels.instances.annotated.v2.ply")
curr_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not curr_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(curr_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(curr_mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
curr_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)


"""
Transform the current mesh into the coordinate system of the new mesh
"""

transformed_mesh_path = transform_to_coordinates(data_dir, curr_scan_id, new_scan_id)
transformed_mesh = o3d.io.read_triangle_mesh(transformed_mesh_path)
#check if it has colours
if not transformed_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(transformed_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(transformed_mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
transformed_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)



"""
New_scan_id mesh display in original coordinate system
"""
pathToMesh = osp.join(data_dir,"scenes", new_scan_id, "labels.instances.annotated.v2.ply")
new_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not new_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(new_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(new_mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
new_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)


"""
Display everything
"""

##visualize everything
vis = o3d.visualization.Visualizer()
vis.create_window(width=800, height=600)

#vis.add_geometry(curr_mesh)
#vis.add_geometry(transformed_mesh)
vis.add_geometry(new_mesh)



vis.run()
vis.destroy_window()


In [97]:
#for a given scan returns the reference, if the scan is reference returns the reference
def get_reference_id(data_dir,scan_id):
    #refscan of a reference is the reference
    if scan3r.is_reference(data_dir, scan_id):
        return scan_id
    
    #open the 3Rjson file
    dir_path = osp.join(data_dir,"files","3RScan.json")
    with open(dir_path, 'r') as file:
        data = json.load(file)

    for item in data:
        if any(scan_id in scan["reference"] for scan in item["scans"]):
            return item["reference"]

    return None